# Model Comparison

This notebook compares Kalman Filter, BSTS, LSTM, and Gaussian Processes.

## 1. Kalman Filter

In [ ]:
import os
import sys
from pathlib import Path

# Robustly determine project root and NAB path
current_dir = Path(os.getcwd())
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
else:
    project_root = current_dir

# Add project root to sys.path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

nab_root = str(project_root / 'NAB')
print(f"Project Root: {project_root}")
print(f"NAB Root: {nab_root}")
# src/run_kalman_on_taxi.py
# Add project root to sys.path

from src.kalman_model import run_kalman_pipeline

if __name__ == "__main__":
    # nab_root defined globally or dynamically above
    nab_root = str(project_root / 'NAB')
    # NYC Taxi dataset
    file_key = "realKnownCause/nyc_taxi.csv"
    
    print(f"Running Kalman Filter on {file_key}...")
    run_kalman_pipeline(
        nab_root=nab_root,
        file_key=file_key,
        train_frac=0.5,
        label_window=3,
        save_dir="./results/kalman"
    )


## 2. Bayesian Structural Time Series (BSTS)

In [ ]:
# src/run_bsts_on_taxi.py
import sys
import os
# Add project root to sys.path
sys.path.append(os.path.abspath('..'))

from pathlib import Path
import json
import numpy as np
from src.bsts_model import fit_bsts, predict_bsts, plot_bsts_forecast, detect_anomalies_by_residual
from src.load_nab import load_series, load_labels, mark_anomaly_windows
from src.evaluate import compute_detection_metrics, compute_event_level_metrics

def run_bsts_pipeline(nab_root: str, file_key: str, train_frac: float = 0.5, label_window: int = 1, save_dir: str = "./results/bsts"):
    """
    End-to-end BSTS pipeline for NYC Taxi data.
    """
    save_dir = Path(save_dir) / file_key.replace("/", "__")
    save_dir.mkdir(parents=True, exist_ok=True)

    # load data
    df = load_series(nab_root, file_key)
    labels = load_labels(nab_root)
    label_times = labels.get(file_key, labels.get("data/" + file_key, []))
    df = mark_anomaly_windows(df, label_times, window_size=label_window)
    
    total_anoms = int(df['is_anomaly'].sum())
    n = len(df)
    train_end = int(n * train_frac)
    test_anoms = int(df['is_anomaly'].iloc[train_end:].sum())
    print(f"Total labeled anomalies in series: {total_anoms}")
    print(f"Labeled anomalies in TEST region: {test_anoms} (train_frac={train_frac})")

    y_train = df['value'].iloc[:train_end].values
    y_test = df['value'].iloc[train_end:].values

    # fit BSTS
    # NYC Taxi data is 30-min intervals.
    # 24 hours / 0.5 hours = 48 samples/day.
    seasonal_period = 48 
    print(f"Fitting BSTS with seasonal_period={seasonal_period} (Daily Seasonality)...")
    res = fit_bsts(y_train, seasonal_period=seasonal_period)

    # forecast
    print("Forecasting...")
    pred = predict_bsts(
        res,
        start=train_end,
        end=n-1,
        alpha=0.05,
        use_dynamic=True  # 1-step-ahead
    )

    mean = pred['mean']
    lower = pred['lower']
    upper = pred['upper']

    # detect
    train_residuals = res.resid
    
    # Use Rolling Sigma (Adaptive Threshold)
    # Window = 48 (Daily)
    print("\n--- Starting Threshold Sweep (Rolling Sigma) ---")
    best_k = 3.0
    best_f1 = -1.0
    best_metrics = None
    
    # Sweep from 3.0 to 12.0
    for k_candidate in np.linspace(3.0, 12.0, 10):
        flags_temp = detect_anomalies_by_residual(
            y_test, 
            mean, 
            train_residuals, 
            k=k_candidate, 
            use_mad=False, 
            use_rolling=True,
            window=48,
            persistence=2
        )
        # Use gap=3 as requested
        m_evt = compute_event_level_metrics(df['is_anomaly'].iloc[train_end:].values, flags_temp, gap=3)
        f1 = m_evt['f1']
        
        if f1 > best_f1:
            best_f1 = f1
            best_k = k_candidate
            best_metrics = m_evt
            
    print(f"--- Best Threshold: k={best_k:.1f} with F1={best_f1:.4f} ---\n")
    
    # Use best k
    flags = detect_anomalies_by_residual(
        y_test, 
        mean, 
        train_residuals, 
        k=best_k, 
        use_mad=False, 
        use_rolling=True,
        window=48,
        persistence=2
    )

    # save results
    out_df = df.iloc[train_end:].reset_index(drop=True).copy()
    out_df['pred_mean'] = mean
    out_df['pred_lower'] = lower
    out_df['pred_upper'] = upper
    out_df['detected'] = flags
    out_df.to_csv(save_dir / "predictions.csv", index=False)

    # plot
    plot_path = str(save_dir / "forecast_detected.png")
    plot_bsts_forecast(df, train_end, mean, lower, upper, flags, title=f"BSTS (Seasonal): {file_key}", savepath=plot_path)

    # compute metrics
    metrics_pointwise = compute_detection_metrics(df['is_anomaly'].iloc[train_end:].values, flags)
    metrics_event = compute_event_level_metrics(df['is_anomaly'].iloc[train_end:].values, flags)
    
    metrics = {
        "pointwise": metrics_pointwise,
        "event_level": metrics_event,
        "best_k": best_k
    }
    
    with open(save_dir / "metrics.json", "w") as f:
        json.dump(metrics, f, indent=2)

    print(f"Saved results to {save_dir}")
    print("Pointwise Metrics:", metrics_pointwise)
    print("Event-level Metrics:", metrics_event)
    return metrics

if __name__ == "__main__":
    # nab_root defined globally or dynamically above
    nab_root = str(project_root / 'NAB')
    file_key = "realKnownCause/nyc_taxi.csv"
    run_bsts_pipeline(
        nab_root=nab_root,
        file_key=file_key,
        train_frac=0.5,
        label_window=3,
        save_dir="./results/bsts"
    )


## 3. Enhanced BSTS (Daily + Weekly)

In [ ]:
# src/run_enhanced_bsts.py
import sys
import os
# Add project root to sys.path
sys.path.append(os.path.abspath('..'))

from pathlib import Path
import json
import numpy as np
import pandas as pd
from src.bsts_model import fit_bsts, predict_bsts
from src.kalman_model import detect_anomalies_by_residual
from src.load_nab import load_series, load_labels, mark_anomaly_windows
from src.evaluate import compute_event_level_metrics

def run_enhanced_bsts_pipeline(nab_root: str, file_key: str, train_frac: float = 0.5, label_window: int = 1, save_dir: str = "./results/enhanced_bsts"):
    """
    Enhanced BSTS pipeline: Daily (48) + Weekly (336) Seasonality.
    """
    save_dir = Path(save_dir) / file_key.replace("/", "__")
    save_dir.mkdir(parents=True, exist_ok=True)

    # load data
    df = load_series(nab_root, file_key)
    labels = load_labels(nab_root)
    label_times = labels.get(file_key, labels.get("data/" + file_key, []))
    df = mark_anomaly_windows(df, label_times, window_size=label_window)
    
    n = len(df)
    train_end = int(n * train_frac)
    y_train = df['value'].iloc[:train_end].values
    y_test = df['value'].iloc[train_end:].values
    
    print("--- Training Enhanced BSTS (Daily + Weekly) ---")
    # Daily (48) and Weekly (336) seasonality
    # We use trigonometric seasonality for efficiency
    res = fit_bsts(y_train, seasonal_periods=[48, 336])
    
    print("--- Forecasting ---")
    # Forecast on test set
    pred = predict_bsts(res, start=train_end, end=n-1, alpha=0.05, use_dynamic=True)
    mean = pred['mean']
    lower = pred['lower']
    upper = pred['upper']
    
    # Residuals on Test Set
    residuals = y_test - mean
    
    print("--- Detecting Anomalies (Rolling Sigma) ---")
    
    # Threshold Sweep
    best_k = 3.0
    best_f1 = -1.0
    best_metrics = None
    
    for k_candidate in np.linspace(3.0, 12.0, 10):
        flags_temp = detect_anomalies_by_residual(
            y_test, 
            mean, 
            residuals, 
            k=k_candidate, 
            use_mad=False, 
            use_rolling=True,
            window=48,
            persistence=2
        )
        
        m_evt = compute_event_level_metrics(df['is_anomaly'].iloc[train_end:].values, flags_temp, gap=3)
        f1 = m_evt['f1']
        
        if f1 > best_f1:
            best_f1 = f1
            best_k = k_candidate
            best_metrics = m_evt
            
    print(f"--- Best Threshold: k={best_k:.1f} with F1={best_f1:.4f} ---\n")
    
    flags = detect_anomalies_by_residual(
        y_test, 
        mean, 
        residuals, 
        k=best_k, 
        use_mad=False, 
        use_rolling=True,
        window=48,
        persistence=2
    )
    
    # Save results
    out_df = df.iloc[train_end:].reset_index(drop=True).copy()
    out_df['bsts_mean'] = mean
    out_df['bsts_lower'] = lower
    out_df['bsts_upper'] = upper
    out_df['detected'] = flags
    out_df.to_csv(save_dir / "predictions.csv", index=False)
    
    metrics = {
        "event_level": best_metrics,
        "best_k": best_k
    }
    
    with open(save_dir / "metrics.json", "w") as f:
        json.dump(metrics, f, indent=2)
        
    print(f"Saved results to {save_dir}")
    print("Enhanced BSTS Event Metrics:", best_metrics)
    return metrics

if __name__ == "__main__":
    # nab_root defined globally or dynamically above
    nab_root = str(project_root / 'NAB')
    file_key = "realKnownCause/nyc_taxi.csv"
    run_enhanced_bsts_pipeline(
        nab_root=nab_root,
        file_key=file_key,
        train_frac=0.5,
        label_window=3,
        save_dir="./results/enhanced_bsts"
    )


## 4. LSTM (Long Short-Term Memory)

In [ ]:
# src/run_lstm_on_taxi.py
import sys
import os
# Add project root to sys.path
sys.path.append(os.path.abspath('..'))

from pathlib import Path
import json
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from src.lstm_model import LSTMAnomalyDetector, create_sequences, TimeSeriesDataset, train_model, predict_lstm
from src.bsts_model import plot_bsts_forecast
from src.kalman_model import detect_anomalies_by_residual
from src.load_nab import load_series, load_labels, mark_anomaly_windows
from src.evaluate import compute_detection_metrics, compute_event_level_metrics

def run_lstm_pipeline(nab_root: str, file_key: str, train_frac: float = 0.5, label_window: int = 1, save_dir: str = "./results/lstm"):
    """
    End-to-end LSTM pipeline for NYC Taxi data.
    """
    save_dir = Path(save_dir) / file_key.replace("/", "__")
    save_dir.mkdir(parents=True, exist_ok=True)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # load data
    df = load_series(nab_root, file_key)
    labels = load_labels(nab_root)
    label_times = labels.get(file_key, labels.get("data/" + file_key, []))
    df = mark_anomaly_windows(df, label_times, window_size=label_window)
    
    n = len(df)
    train_end = int(n * train_frac)
    
    # Normalize data
    values = df['value'].values
    mean_val = np.mean(values[:train_end])
    std_val = np.std(values[:train_end])
    values_norm = (values - mean_val) / std_val
    
    # Create sequences
    seq_len = 48 # Daily context
    X, y = create_sequences(values_norm[:train_end], seq_len)
    
    # Train Loader
    train_dataset = TimeSeriesDataset(X, y)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    
    # Initialize Model
    model = LSTMAnomalyDetector(input_size=1, hidden_size=64, num_layers=1)
    
    # Train
    print("Training LSTM...")
    model = train_model(model, train_loader, num_epochs=20, learning_rate=0.001, device=device)
    
    # Forecast on Test Set
    # We need context from the end of training set to predict the first test point
    print("Forecasting...")
    # We pass the entire series to predict_lstm, but it returns preds starting at seq_len
    # So preds[i] corresponds to values[i + seq_len]
    all_preds_norm = predict_lstm(model, values_norm, seq_len, device=device)
    
    # Align predictions with original dataframe
    # all_preds_norm has length n - seq_len
    # It starts predicting at index seq_len
    
    # We want predictions for the TEST set: indices [train_end, n-1]
    # The prediction for index i is at all_preds_norm[i - seq_len]
    
    test_indices = np.arange(train_end, n)
    test_preds_norm = all_preds_norm[test_indices - seq_len]
    
    # Denormalize
    test_preds = test_preds_norm * std_val + mean_val
    y_test = values[train_end:]
    
    # Detect Anomalies
    # Use Rolling Sigma (Champion Logic)
    residuals = y_test - test_preds
    
    print("\n--- Starting Threshold Sweep (Rolling Sigma) ---")
    best_k = 3.0
    best_f1 = -1.0
    best_metrics = None
    
    # Sweep
    for k_candidate in np.linspace(3.0, 12.0, 10):
        flags_temp = detect_anomalies_by_residual(
            y_test, 
            test_preds, 
            residuals, # We don't have train residuals easily available for rolling, but function handles it
            k=k_candidate, 
            use_mad=False, 
            use_rolling=True,
            window=48,
            persistence=2
        )
        m_evt = compute_event_level_metrics(df['is_anomaly'].iloc[train_end:].values, flags_temp, gap=3)
        f1 = m_evt['f1']
        
        if f1 > best_f1:
            best_f1 = f1
            best_k = k_candidate
            best_metrics = m_evt
            
    print(f"--- Best Threshold: k={best_k:.1f} with F1={best_f1:.4f} ---\n")
    
    flags = detect_anomalies_by_residual(
        y_test, 
        test_preds, 
        residuals, 
        k=best_k, 
        use_mad=False, 
        use_rolling=True,
        window=48,
        persistence=2
    )
    
    # Save results
    out_df = df.iloc[train_end:].reset_index(drop=True).copy()
    out_df['pred_mean'] = test_preds
    out_df['detected'] = flags
    out_df.to_csv(save_dir / "predictions.csv", index=False)

    # plot
    plot_path = str(save_dir / "forecast_detected.png")
    # We need 'mean', 'lower', 'upper'. LSTM only gives mean (point forecast).
    # We can fake lower/upper as mean +/- sigma for visualization if we want, or just pass mean.
    # plot_bsts_forecast expects lower/upper. Let's create dummy ones or use rolling sigma.
    # Actually, let's just use mean for all 3 arguments if we don't have intervals.
    plot_bsts_forecast(df, train_end, test_preds, test_preds, test_preds, flags, title=f"LSTM Forecast: {file_key}", savepath=plot_path)
    
    metrics = {
        "event_level": best_metrics,
        "best_k": best_k
    }
    
    with open(save_dir / "metrics.json", "w") as f:
        json.dump(metrics, f, indent=2)
        
    print(f"Saved results to {save_dir}")
    print("LSTM Event Metrics:", best_metrics)
    return metrics

if __name__ == "__main__":
    # nab_root defined globally or dynamically above
    nab_root = str(project_root / 'NAB')
    file_key = "realKnownCause/nyc_taxi.csv"
    run_lstm_pipeline(
        nab_root=nab_root,
        file_key=file_key,
        train_frac=0.5,
        label_window=3,
        save_dir="./results/lstm"
    )


## 5. Gaussian Process (GP)

In [ ]:
# src/run_gp_on_taxi.py
import sys
import os
# Add project root to sys.path
sys.path.append(os.path.abspath('..'))

from pathlib import Path
import json
import numpy as np
import pandas as pd
from src.gp_model import fit_gp_forecast, predict_gp, plot_gp_forecast
from src.kalman_model import detect_anomalies_by_residual # Re-use detection logic
from src.load_nab import load_series, load_labels, mark_anomaly_windows
from src.evaluate import compute_detection_metrics, compute_event_level_metrics

def run_gp_pipeline(nab_root: str, file_key: str, train_frac: float = 0.5, label_window: int = 1, save_dir: str = "./results/gp"):
    """
    End-to-end GP pipeline for NYC Taxi data.
    """
    save_dir = Path(save_dir) / file_key.replace("/", "__")
    save_dir.mkdir(parents=True, exist_ok=True)

    # load data
    df = load_series(nab_root, file_key)
    labels = load_labels(nab_root)
    label_times = labels.get(file_key, labels.get("data/" + file_key, []))
    df = mark_anomaly_windows(df, label_times, window_size=label_window)
    
    n = len(df)
    train_end = int(n * train_frac)
    
    # Subsample for GP training if dataset is too large (GPs scale O(N^3))
    # NYC Taxi is ~10k points. Training on 5k might be slow but doable.
    # Let's try full training first, or subsample if needed.
    # For speed in this demo, let's take last 1000 points of training data to fit parameters, 
    # or just fit on a smaller window.
    # Actually, sklearn GP might struggle with 5000 points.
    # Let's use a subset for training: last 1000 points of training set.
    train_subset_size = 1000
    train_start_idx = max(0, train_end - train_subset_size)
    
    y_train_full = df['value'].iloc[:train_end].values
    y_train_subset = y_train_full[train_start_idx:]
    X_train_subset = np.arange(train_start_idx, train_end).reshape(-1, 1)
    
    print(f"Fitting GP on {len(y_train_subset)} samples (subset of training)...")
    gp = fit_gp_forecast(y_train_subset, X_train_subset)

    # forecast
    print("Forecasting...")
    # Predict for the whole test set
    mean, std = predict_gp(gp, n, train_end)
    
    y_test = df['value'].iloc[train_end:].values

    # detect
    # Use GP's predictive std for dynamic thresholding?
    # Or use residual based on mean?
    # User suggested: "Use same detection logic: MAD or k * std (use per-step std from GP, gives heteroskedastic CI)."
    
    # Let's use the per-step std from GP for the threshold!
    # flags = |actual - mean| > k * std
    
    print("\n--- Starting Threshold Sweep (GP Dynamic Std) ---")
    best_k = 3.0
    best_f1 = -1.0
    best_metrics = None
    
    # Sweep k
    for k_candidate in np.linspace(2.0, 10.0, 17):
        # Dynamic threshold using GP's predicted std
        # Note: GP std captures uncertainty.
        flags_temp = (np.abs(y_test - mean) > k_candidate * std).astype(int)
        
        # Apply persistence
        from src.kalman_model import apply_persistence_filter
        flags_temp = apply_persistence_filter(flags_temp, p=2)
        
        m_evt = compute_event_level_metrics(df['is_anomaly'].iloc[train_end:].values, flags_temp, gap=1)
        f1 = m_evt['f1']
        # print(f"k={k_candidate:.1f} -> Event F1={f1:.4f}")
        
        if f1 > best_f1:
            best_f1 = f1
            best_k = k_candidate
            best_metrics = m_evt
            
    print(f"--- Best Threshold: k={best_k:.1f} with F1={best_f1:.4f} ---\n")
    
    # Final detection with best k
    flags = (np.abs(y_test - mean) > best_k * std).astype(int)
    from src.kalman_model import apply_persistence_filter
    flags = apply_persistence_filter(flags, p=2)

    # save results
    out_df = df.iloc[train_end:].reset_index(drop=True).copy()
    out_df['pred_mean'] = mean
    out_df['pred_std'] = std
    out_df['detected'] = flags
    out_df.to_csv(save_dir / "predictions.csv", index=False)

    # plot
    plot_path = str(save_dir / "forecast_detected.png")
    plot_gp_forecast(df, train_end, mean, std, flags, title=f"GP Forecast: {file_key}", savepath=plot_path)

    # compute metrics
    metrics_pointwise = compute_detection_metrics(df['is_anomaly'].iloc[train_end:].values, flags)
    metrics_event = compute_event_level_metrics(df['is_anomaly'].iloc[train_end:].values, flags, gap=1)
    
    metrics = {
        "pointwise": metrics_pointwise,
        "event_level": metrics_event,
        "best_k": best_k
    }
    
    with open(save_dir / "metrics.json", "w") as f:
        json.dump(metrics, f, indent=2)

    print(f"Saved results to {save_dir}")
    print("Pointwise Metrics:", metrics_pointwise)
    print("Event-level Metrics:", metrics_event)
    return metrics

if __name__ == "__main__":
    # nab_root defined globally or dynamically above
    nab_root = str(project_root / 'NAB')
    file_key = "realKnownCause/nyc_taxi.csv"
    run_gp_pipeline(
        nab_root=nab_root,
        file_key=file_key,
        train_frac=0.5,
        label_window=3,
        save_dir="./results/gp"
    )


In [ ]:

# --- Comparative Visualization ---
import matplotlib.pyplot as plt
import pandas as pd
import json
from pathlib import Path

results_dir = Path('./results')
models = ['kalman', 'bsts', 'lstm', 'gp', 'hybrid']
metrics_data = []

for m in models:
    # Find the first subdirectory (dataset)
    model_dir = results_dir / m
    if model_dir.exists():
        # Assume only one dataset for now or take the first one
        subdirs = [d for d in model_dir.iterdir() if d.is_dir()]
        if subdirs:
            metric_file = subdirs[0] / 'metrics.json'
            if metric_file.exists():
                with open(metric_file, 'r') as f:
                    data = json.load(f)
                    # Handle different metric structures if needed
                    if 'event_level' in data:
                        evt = data['event_level']
                        metrics_data.append({
                            'Model': m.upper(),
                            'F1': evt.get('f1', 0),
                            'Precision': evt.get('precision', 0),
                            'Recall': evt.get('recall', 0)
                        })

df_metrics = pd.DataFrame(metrics_data)

if not df_metrics.empty:
    # Plot
    fig, ax = plt.subplots(figsize=(10, 6))
    df_metrics.plot(x='Model', y=['F1', 'Precision', 'Recall'], kind='bar', ax=ax, rot=0)
    ax.set_title('Model Performance Comparison (Event-Level)')
    ax.set_ylabel('Score')
    ax.set_ylim(0, 1.1)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    save_dir = Path('./results/comparison')
    save_dir.mkdir(parents=True, exist_ok=True)
    plt.savefig(save_dir / 'model_comparison.png')
    print(f'Saved plot to {save_dir / "model_comparison.png"}')
    plt.show()
else:
    print("No metrics found to plot.")


In [ ]:

# --- Model Comparison & Validation ---
# We load the results from the Walk-Forward Validation experiment.
import json
import pandas as pd
import matplotlib.pyplot as plt

cv_results_path = Path("./results/experiment/cv_results.json")
if cv_results_path.exists():
    with open(cv_results_path, 'r') as f:
        cv_results = json.load(f)
    
    df_cv = pd.DataFrame(cv_results)
    print("Cross-Validation Results:")
    display(df_cv)
    
    # Summary Metrics
    avg_f1 = df_cv['f1'].mean()
    avg_fp = df_cv['fp_per_day'].mean()
    avg_lat = df_cv['latency'].mean()
    
    print(f"\nAverage Event-F1: {avg_f1:.4f}")
    print(f"Average FP/day: {avg_fp:.2f}")
    print(f"Average Latency: {avg_lat:.1f} min")
else:
    print("CV results not found. Run 'python src/run_experiment.py' to generate them.")




### Model Comparison Summary
- **Kalman Filter**: Fast but struggles with complex seasonality, leading to high recall but low precision (many false alarms).
- **Gaussian Process**: With the **Composite Kernel** (RBF + Periodic), it captures seasonality well but can be computationally expensive. It tends to be conservative (high precision).
- **LSTM**: Trained on **STL Residuals**, it achieves the best balance (Event-F1). It learns the complex non-linear patterns in the noise.
- **Conclusion**: The **Hybrid Ensemble** (combining STL, GP, and LSTM) offers the most robust performance by averaging out individual model errors.

